# Supermarket Sales

**Objective:** Obtain a dataset ready for exploratory analysis (EDA).

**Scope:** This notebook focuses solely on structural data cleaning:
- data types
- encoding
- basic normalization


No exploratory data analysis (EDA) or row filtering is performed.

In [2]:
import pandas as pd
import numpy as np

In [3]:
raw = pd.read_csv("SuperMarketAnalysis.csv")
print(f"Initial shape of the dataset: {raw.shape}")

df = raw.copy(deep=True)

Initial shape of the dataset: (1000, 17)


# Snapshot of the data

In [ ]:
df.info()


Branch
Alex     340
Cairo    332
Giza     328
Name: count, dtype: int64

In [5]:
df.describe()

,Unit price,Quantity,Tax 5%,Sales,cogs,gross margin percentage,gross income,Rating
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000
mean,55.672130,5.510000,15.379369,322.966749,307.58738,4.761905,15.379369,6.97270
std,26.494628,2.923431,11.708825,245.885335,234.17651,0.000000,11.708825,1.71858
min,10.080000,1.000000,0.508500,10.678500,10.17000,4.761905,0.508500,4.00000
25%,32.875000,3.000000,5.924875,124.422375,118.49750,4.761905,5.924875,5.50000
50%,55.230000,5.000000,12.088000,253.848000,241.76000,4.761905,12.088000,7.00000
75%,77.935000,8.000000,22.445250,471.350250,448.90500,4.761905,22.445250,8.50000
max,99.960000,10.000000,49.650000,1042.650000,993.00000,4.761905,49.650000,10.00000


In [6]:
df.head(4)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,1:08:00 PM,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29:00 AM,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,1:23:00 PM,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,8:33:00 PM,Ewallet,465.76,4.761905,23.2880,8.4


## Data types

In [7]:
df.dtypes

Invoice ID                  object
Branch                      object
City                        object
Customer type               object
Gender                      object
Product line                object
Unit price                 float64
Quantity                     int64
Tax 5%                     float64
Sales                      float64
Date                        object
Time                        object
Payment                     object
cogs                       float64
gross margin percentage    float64
gross income               float64
Rating                     float64
dtype: object

Several columns should be of numeric types. Therefore, data types are converted.

In [8]:
numeric_cols = ["Unit price", "Quantity", "Tax 5%", 
                "Sales", "cogs", "gross margin percentage",
                    "gross income", "Rating"]

for col in numeric_cols:
    df[col] =pd.to_numeric(df[col], errors="coerce")

    assert df[col].dtype in [np.float64, np.int64], \
        f"Column {col} is not numeric after conversion."
    
print("All specified columns have been successfully converted to numeric types.")
df.info()


All specified columns have been successfully converted to numeric types.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Sales                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  cogs  

### Treatment of the columns Date and Time
Columns `Date` and `Time` are consolidated into a single object `Datetime`, to facilitate temporal analysis and chronological continuity.

In [9]:
if "Date" in df.columns and "Time" in df.columns:
    df["Datetime"] = pd.to_datetime(df["Date"] + " " + \
                                     df["Time"], errors="coerce")

assert df["Datetime"].dtype == "datetime64[ns]", \
    "Datetime column is not of type datetime64[ns] after conversion."

# The original "Date" and "Time" columns are no longer needed,
# so we can drop them.
df.drop(columns=["Date", "Time"], inplace=True, errors="ignore")

print(f"Shape of the dataset after Datetime inclusion and cleaning: {df.shape}")


/tmp/ipykernel_1030/3585069625.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Datetime"] = pd.to_datetime(df["Date"] + " " + \


Shape of the dataset after Datetime inclusion and cleaning: (1000, 16)


## Strings cleaning
The white spaces from columns of string type are removed. 


In [10]:
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].apply(lambda s: s.str.strip())

## Nan overview
The data is analysed to see how much data is missing. 

In [11]:
na_ratio = df.isna().mean().sort_values(ascending=False)
print(na_ratio)

Invoice ID                 0.0
Branch                     0.0
City                       0.0
Customer type              0.0
Gender                     0.0
Product line               0.0
Unit price                 0.0
Quantity                   0.0
Tax 5%                     0.0
Sales                      0.0
Payment                    0.0
cogs                       0.0
gross margin percentage    0.0
gross income               0.0
Rating                     0.0
Datetime                   0.0
dtype: float64


It can be seen that this data frame has no NaN values.

## Verifying the presence of duplicates

In [12]:
print(f"Number of duplicate rows: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)
print(f"Shape of the dataset after removing duplicates: {df.shape}")

Number of duplicate rows: 0
Shape of the dataset after removing duplicates: (1000, 16)


## Changing columns of type object to category
Several columns in the dataset were converted from the type `object` to the Pandas' type `category` , to reduce the use of RAM memory (~70% for these columns). 

In this small dataset the impact is reduced, but when we have hundred of thousands or millons rows, the reduction in memory use can be drastic.



In [ ]:
# Customer type column:
df["Customer type"] = df["Customer type"].replace({"Member": "M", "Normal": "N"})
# Now we convert the "Customer type" column to a categorical type for better memory efficiency.
df["Customer type"] = df["Customer type"].astype("category")

assert df["Customer type"].dtype.name == "category", \
    "Customer type column is not of type 'category' after conversion."

# Gender column:
df["Gender"] = df["Gender"].replace({"Female": "F", "Male": "M"})
df["Gender"] = df["Gender"].astype("category")

assert df["Gender"].dtype.name == "category", \
    "Gender column is not of type 'category' after conversion."

# Payment column:
df["Payment"] = df["Payment"].astype("category")
assert df["Payment"].dtype.name == "category", \
    "Payment column is not of type 'category' after conversion."


# City column:
df['City'] = df['City'].str.capitalize() # Standardize by capitalizing the first letter
df['City'] = df['City'].astype('category')
assert df['City'].dtype.name == 'category', \
    "City column is not of type 'category' after conversion."

# Branch column:
df['Branch'] = df['Branch'].str.capitalize() # Standardize by capitalizing the first letter
df['Branch'] = df['Branch'].astype('category')
assert df['Branch'].dtype.name == 'category', \
    "Branch column is not of type 'category' after conversion."

# Product line column:
df['Product line'] = df['Product line'].str.capitalize() # Standardize by capitalizing first letter
df['Product line'] = df['Product line'].astype('category')

Product line
Fashion accessories       178
Food and beverages        174
Electronic accessories    170
Sports and travel         166
Home and lifestyle        160
Health and beauty         152
Name: count, dtype: int64

## Validation of several features of the dataset

- The invoice IDs must exist and be unique for each row.
- Numeric columns must contain values greater or equal than zero.
- Raiting columns must contain values between 1 and 10.
- Integrity check: financial totals were validated for consistency. Any rounding discrepancies were analyzed.


In [14]:
def validate(df):
    assert df["Invoice ID"].is_unique, "Invoice ID column contains duplicate values."
    assert df['Invoice ID'].nunique() == len(df), "Integrity error: There are duplicate or corrupt IDs"
    
    for col in numeric_cols:
        assert df[col].ge(0).all(), \
            f"Column {col} contain negative values, which is unexpected for this dataset."
    assert df["Rating"].between(1, 10).all(), \
        "Rating column contains values outside the expected range of 1 to 10."


    # Column Sales validation: Sales should be equal to COGS + Tax
    expected_sales = df['cogs'] + df['Tax 5%']
    
    # Comparamos con un margen de error por redondeo
    is_valid = np.allclose(df['Sales'], expected_sales, atol=0.01)
    assert is_valid, "Integrity error: Sales != COGS + Tax 5%"


    print("✅ Successfully validated the dataset. All checks passed.")

validate(df)

print(f"Final shape of the cleaned dataset: {df.shape}")

✅ Successfully validated the dataset. All checks passed.
Final shape of the cleaned dataset: (1000, 16)


## Quality report

In [18]:
def generate_quality_report(df):
    report = pd.DataFrame({
        "Dtype": df.dtypes,
        "Total Rows": len(df),
        "Non-Null Count": df.count(),
        "Missing (%)": (df.isna().mean() * 100).round(2),
        "Unique Values": df.nunique(),
        "Zero Values": (df == 0).sum(),
        "Negatives": df.select_dtypes(include=['number']).lt(0).sum()
    })
    
    # Basic statistics only for numeric columns
    desc = df.describe().T[['min', 'mean', 'max']].round(2)
    report = report.join(desc, how='left')
    
    return report

# Generate and print the quality report
quality_summary = generate_quality_report(df)
print(quality_summary)

                                  Dtype  Total Rows  Non-Null Count  \
Branch                           object        1000            1000   
City                             object        1000            1000   
Customer type                  category        1000            1000   
Datetime                 datetime64[ns]        1000            1000   
Gender                         category        1000            1000   
Invoice ID                       object        1000            1000   
Payment                        category        1000            1000   
Product line                     object        1000            1000   
Quantity                          int64        1000            1000   
Rating                          float64        1000            1000   
Sales                           float64        1000            1000   
Tax 5%                          float64        1000            1000   
Unit price                      float64        1000            1000   
cogs  

## Exporting the cleaned dataset


In [ ]:
# The final data and the quality report are saved in an Excel file with two sheets
final_file = "SupermarketSales_Cleaned.xlsx"

with pd.ExcelWriter(final_file, engine='xlsxwriter') as writer:
    # Sheet 1: cleaned data ready to use
    df.to_excel(writer, sheet_name='Clean Data', index=False)
    
    # Sheet 2: quality report
    quality_summary.to_excel(writer, sheet_name='Quality Report')

print(f"Data and quality report saved succesfuly in {final_file}")

Data and quality report saved succesfuly in SupermarketSales.xlsx
